<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/notebooks/processed/ct_preprocessing_ubs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
===================================================
Author: Chiaki Tachikawa
Role: Data Science Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/chiaki-tachikawa
Date: 2025-03-03
Version: 1.1

Description:
    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:
    - Defining and applying a `preprocessor` function to clean and tokenize text data.
    - Reading and preprocessing various CSV files containing transcript data.
    - Segmenting text by bank name
    - Pairing question and answer by traditional data engineering techniques.
    - Cleaning Texts
    - Reorganising Data Frame
    - Exporting the preprocessed data to new CSV files for further analysis.

===================================================
"""

'\n===================================================\nAuthor: Chiaki Tachikawa\nRole: Data Science Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/chiaki-tachikawa\nDate: 2025-03-03\nVersion: 1.1\n\nDescription:\n    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:\n    - Defining and applying a `preprocessor` function to clean and tokenize text data.\n    - Reading and preprocessing various CSV files containing transcript data.\n    - Segmenting text by bank name\n    - Pairing question and answer by traditional data engineering techniques.\n    - Cleaning Texts\n    - Reorganising Data Frame\n    - Exporting the preprocessed data to new CSV files for further analysis.\n\n===================================================\n'

# **Library**

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from google.colab import drive
import openai
import json
import os
from google.colab import userdata
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Function**

**preprocessor function** : The function modifies the DataFrame data in place, adding two new columns (col1 and col2) with preprocessed text.


Input:
  - name of dataframe
  - name of column which contains the text to clean
  - name of column which is tokenized
  - name of column which is cleaned

In [ ]:
#create function to preprocess data
def preprocessor (data, col, col1,col2):
  #Copy col1umn
  data[col1]=data[col]
  data[col2]=data[col]


  #Adding column1
  #Lower the lettercase
  data[col1] = data[col1].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col1] = data[col1].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #Tokenize the word
  data[col1] = data[col1].apply(nltk.word_tokenize)

  #Remove numbers
  data[col1] = data[col1].apply(lambda x: [word for word in x if not word.isdigit()])

  #remove symbol from comments
  data[col1] = data[col1].apply(lambda x: [word for word in x if x!=""])

  #remove short word
  data[col1] = data[col1].apply(lambda x: [word for word in x if len(word)>2])

  #remove symbols
  data[col1] = data[col1].apply (lambda x: [re.sub(r"[^a-z]", "", word) for word in x])

  #lemmatization
  lemmatizer = WordNetLemmatizer()
  data[col1] = data[col1].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])



  #Adding column2
  #Lower the lettercase
  data[col2] = data[col2].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col2] = data[col2].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #remove symbols
  data[col2] = data[col2].apply (lambda x: [re.sub(r"[.,'?]", "", x)])

  return


In [ ]:
# Function to extract names
def extract_name(full_string):
    return full_string.split(',')[0]

## **Structuring Data Frame**
Ensuring that each row has utterance from one analyst and exective and categorised by bank name to pair question and answer later.

In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Defining qa_data
ubs_qa_data = pd.read_csv("/content/drive/MyDrive/BOE/bank_of_england/data/cleansed/ubs_qna_section.csv")
#Check the imported data
#ubs_qa_data.head()
print(ubs_qa_data)

               speaker      job_title  \
0              Unknown            NaN   
1    Sergio P. Ermotti            NaN   
2         Chris Hallam  Goldman Sachs   
3    Sergio P. Ermotti            NaN   
4      Sarah Youngwood            NaN   
..                 ...            ...   
304  Sergio P. Ermotti            NaN   
305       Todd Tuckner            NaN   
306       Todd Tuckner            NaN   
307  Sergio P. Ermotti            NaN   
308  Sergio P. Ermotti            NaN   

                                             utterance        call_date  \
0    Chis Hallam, Goldman Sachs Yes. Good morning, ...    25 April 2023   
1    Okay. Thank you. On capital requirements, you ...    25 April 2023   
2    Very clear. Thanks. Kian Abouhossein, JPMorgan...    25 April 2023   
3    So, Sarah, take the first question. I'll take ...    25 April 2023   
4    So, when we give you the 74%, we focused inten...    25 April 2023   
..                                                 ...   

In [ ]:
"""
This code searches for Analyst / Exective name and their bank name in the 'utterance' column of a DataFrame and stores any matches in a new 'dummy' column. If no matches are found, the 'dummy' column remains None for that row.

"""
#Defining Analyst name and Bank Name with regex
pattern = r'\b[A-Z][a-z]+ [A-Z][a-z]+, [A-Z][A-Za-z]+'

#Looking for Analyst name and bank name from utterance to store it in dummy column
ubs_qa_data["dummy"]=None
for i in range(len(ubs_qa_data)):
  matches = re.findall(pattern, str(ubs_qa_data['utterance'][i]))
  if matches:
    ubs_qa_data.at[i, 'dummy'] = matches
  else:
    continue

#Defining lists of Exective names from UBS
exective_name = ["Sergio P. Ermotti","Sarah Youngwood	"]
exect_name = r'\b(Sergio P\. Ermotti|Sarah Youngwood)\b'

#Looking for Exective name from utterance to store it in ex_dummy column
ubs_qa_data["ex_dummy"]=None
for i in range(len(ubs_qa_data)):
  matches1 = re.findall(exect_name, str(ubs_qa_data["utterance"][i]))
  if matches1:
    ubs_qa_data.at[i,"ex_dummy"]=matches1
    ubs_qa_data.loc[i,"ex_dummy"].append("UBS")
  else:
    continue


In [ ]:
#Check if matched analyst name and exective name are populated in dummy and ex_dummy columns
#ubs_qa_data.head()

In [ ]:
"""
This code searches for Analyst/Exective name and their bank name in the 'utterance' column of a DataFrame, updates the 'speaker' column for the first row with matches, and splits the 'utterance' and inserts a new row for subsequent matches. If no matches are found, the loop continues to the next row.
Here are the steps in the code:
1, Defining matches with Bank name and Exective name in each utterance.
2, Create 3 conditions as follows.
  a, if it is the first row
    > Added Analyst and Bank name in speaker column and job_title column
  b, if there is bank name in the utterance
    I, when there is one match in the utterance
    > Created new row between the current and next index to split the utterance by each analyst
    II, when there are two matches in the utterance
    > Create new multiple rows between the current and next index to split the utterance by each analyst
  c, if there is Exective name in the utterance
    > Create a new row between the current and next index to split the utterance by each exective
"""


for  i in ubs_qa_data.index:
  matches = re.findall(pattern, str(ubs_qa_data['utterance'][i]))
  matches1 = re.findall(exect_name, str(ubs_qa_data['utterance'][i]))
  if matches and i==0:
    ubs_qa_data.at[i, 'speaker'] = matches
    ubs_qa_data.at[i,"job_title"]= matches
  elif matches:
    if len(ubs_qa_data["dummy"][i]) <2:
      new_index=i+0.5
      parts1 = [part.strip() for part in ubs_qa_data['utterance'][i].split(matches[0])]
      ubs_qa_data.at[i, 'utterance'] = parts1[0]
      ubs_qa_data.loc[new_index] = {"speaker":matches,"job_title":matches,"utterance":parts1[1], "call_date":ubs_qa_data["call_date"][i], "financial_quarter":ubs_qa_data["financial_quarter"][i],"source_file":ubs_qa_data["source_file"][i], "dummy":None}
    else:
      parts1 = [part.strip() for part in ubs_qa_data['utterance'][i].split(matches[0])]
      parts2 = [part.strip() for part in ubs_qa_data['utterance'][i].split(matches[1])]
      parts3 = [part.strip() for part in parts1[1].split(matches[1])]
      ubs_qa_data.at[i, 'utterance'] = parts1[0]
      ubs_qa_data.loc[i+0.2] = {"speaker":matches[0],"job_title":matches[0],"utterance":parts3[0], "call_date":ubs_qa_data["call_date"][i], "financial_quarter":ubs_qa_data["financial_quarter"][i],"source_file":ubs_qa_data["source_file"][i], "dummy":None}
      ubs_qa_data.loc[i+0.4] = {"speaker":matches[1],"job_title":matches[1],"utterance":parts2[1], "call_date":ubs_qa_data["call_date"][i], "financial_quarter":ubs_qa_data["financial_quarter"][i],"source_file":ubs_qa_data["source_file"][i], "dummy":None}
  elif matches1:
    new_index=i+0.5
    parts2 = [part.strip() for part in ubs_qa_data['utterance'][i].split(matches1[0])]
    ubs_qa_data.at[i, 'utterance'] = parts2[0]
    ubs_qa_data.loc[new_index] = {"speaker":ubs_qa_data["ex_dummy"][i],"job_title":"UBS","utterance":parts2[1], "call_date":ubs_qa_data["call_date"][i], "financial_quarter":ubs_qa_data["financial_quarter"][i],"source_file":ubs_qa_data["source_file"][i], "dummy":None}
  else:
    continue


In [ ]:
"""
Reset index due to new rows
"""
ubs_qa_data=ubs_qa_data.sort_index().reset_index(drop=True)

In [ ]:
"""
This code checks if the 'speaker' column contains a list, splits the first element of the list at the comma, and updates the 'speaker' and 'job_title' columns accordingly. If the 'speaker' is not a list, the loop continues to the next row.

"""
for i in range(len(ubs_qa_data)):
  if isinstance(ubs_qa_data['job_title'][i], list):
    parts = [part.strip() for part in ubs_qa_data['speaker'][i][0].split(',')]
    ubs_qa_data.at[i, 'speaker'] = parts[0]
    ubs_qa_data.at[i, 'job_title'] = parts[1]

  else:
    continue



In [ ]:
#Check if utterance was split correctly, and new rows were created when there is matched analyst name in utterance.
#ubs_qa_data.head()

In [ ]:
"""
Row 56: The text was not populated in the utterance due to irregular simbol in the original text, therefore, this needs to be concanated with the text in the next row.
"""
text = str(ubs_qa_data["speaker"][57]) +str(ubs_qa_data["job_title"][57]) +str(ubs_qa_data["utterance"][57])
ubs_qa_data.at[56,"utterance"]=text
ubs_qa_data=ubs_qa_data.drop(index=57)
ubs_qa_data.reset_index(drop=True, inplace=True)

"""
Row 72: Some words were detected as "Analyst and Bank name" in the text, therefore, this needs to be concanated with the text in the next row.
"""
text1 = str(ubs_qa_data["speaker"][73]) +str(ubs_qa_data["job_title"][73]) +str(ubs_qa_data["utterance"][73])
ubs_qa_data.at[72,"utterance"]= ubs_qa_data["utterance"][72]+text1
ubs_qa_data=ubs_qa_data.drop(index=73)
ubs_qa_data.reset_index(drop=True, inplace=True)

"""
Row 2: Oringinal text shows Goldman Sachs, therefore, this needs to be concanated with the text in the next row.
"""
ubs_qa_data.at[2,"job_title"]="Goldman"

#Remove last row
ubs_qa_data=ubs_qa_data.drop(index=len(ubs_qa_data)-1)
ubs_qa_data.reset_index(drop=True, inplace=True)

#set speaker in row 30 and 31
ubs_qa_data.at[30,"job_title"]= "RBC"
ubs_qa_data.at[31,"job_title"]="Barclays"

In [ ]:
#Check if all bugs are fixed
#ubs_qa_data[50:80]

In [ ]:
"""
This code snippet categorizes rows in a DataFrame ubs_qa_data based on the job_title column.
This effectively groups consecutive rows with the same bank name together to help segmenting question and answer by bank later.
"""
ubs_qa_data["category"]=None
current_bank=ubs_qa_data["job_title"][0]
for i in range(len(ubs_qa_data)):
  if current_bank == ubs_qa_data["job_title"][i]:
    ubs_qa_data.at[i,"category"]=current_bank
  elif current_bank!=ubs_qa_data["job_title"][i] and ubs_qa_data["job_title"][i] is np.NaN:
    ubs_qa_data.at[i,"category"]=current_bank
  else:
    current_bank=ubs_qa_data["job_title"][i]
    ubs_qa_data.at[i,"category"]=current_bank


In [ ]:
"""
add ubs in nan in job title
"""
for i in range(len(ubs_qa_data)):
  if ubs_qa_data["job_title"][i] is np.NaN:
    ubs_qa_data.at[i,"job_title"]="UBS"

In [ ]:
"""
Remove Caustionary statements at the end of transcript in each quarter.

"""
statement = r'\b(Cautionary Statement Regarding|Cautionary statement regarding)\b'
for i in range(len(ubs_qa_data)):
  matches = re.findall(statement, str(ubs_qa_data["utterance"][i]))
  if matches:
    parts = [part.strip() for part in ubs_qa_data['utterance'][i].split(matches[0])]
    ubs_qa_data.at[i, 'utterance'] = parts[0]
  else:
    continue

In [ ]:
print(f'Here is the number of null value in the dataframe.: \n{ubs_qa_data.isnull().sum()}')

Here is the number of null value in the dataframe.: 
speaker                0
job_title              0
utterance              0
call_date              0
financial_quarter      0
source_file            0
dummy                309
ex_dummy             364
category               0
dtype: int64


In [ ]:
#Check if the data was segmented by bank
#ubs_qa_data[130:150]

# **Pairing Question and Answer:**


In [ ]:
"""
Given that the ex_dummy column is no longer needed, next row's financial_quarter column is copied to the ex_dummy column.
"""
for i in range(len(ubs_qa_data)-1):
  ubs_qa_data.at[i,"ex_dummy"]=ubs_qa_data.at[i+1,"financial_quarter"]

In [ ]:
#ubs_qa_data.head()

In [ ]:
"""
This code groups rows by bank name, and splits question and answer into separate columns.
Here are the steps;
1, Creating new columns to pair question and answer, and defining current bank name
2, Looping through each row in the DataFrame
3, Populating values in question and answer columns based on the following conditions:
  a, if the current row's financial quarter is same as the next row's.
  b, if the current row is the last row in the category.
  Otherwise, each row utterance concatenates the current answer or question
"""

ubs_qa_data["question"] = None
ubs_qa_data["answer"] = None
ubs_qa_data["analyst"] = None
ubs_qa_data["exective"] = None
current_bank=ubs_qa_data["category"][0]
analyst=[]
exective=[]
question=""
answer=""

for i in range(len(ubs_qa_data)):
  if ubs_qa_data["ex_dummy"][i]!=ubs_qa_data["financial_quarter"][i] and ubs_qa_data["job_title"][i]=="UBS":
      exective.append(str(ubs_qa_data["speaker"][i]))
      answer += str(ubs_qa_data["utterance"][i])
      ubs_qa_data.at[i,"question"]=question
      ubs_qa_data.at[i,"answer"]=answer
      ubs_qa_data.at[i,"analyst"]=analyst
      ubs_qa_data.at[i,"exective"]=exective
      exective=[]
      analyst=[]
      answer = ""
      question = ""
      ubs_qa_data.at[i,"category"]=current_bank
      if i<len(ubs_qa_data)-1:
        current_bank = ubs_qa_data["job_title"][i+1]
  else:
    if ubs_qa_data["job_title"][i]!="UBS" and current_bank==ubs_qa_data["category"][i]:
      question += str(ubs_qa_data["utterance"][i])
      analyst.append(str(ubs_qa_data["speaker"][i]))
    elif ubs_qa_data["job_title"][i]=="UBS" and current_bank==ubs_qa_data["category"][i]:
      answer += str(ubs_qa_data["utterance"][i])
      exective.append(str(ubs_qa_data["speaker"][i]))
    elif ubs_qa_data["job_title"][i]!="UBS" and current_bank!=ubs_qa_data["category"][i]:
      current_bank=ubs_qa_data["job_title"][i]
      analyst.append(str(ubs_qa_data["speaker"][i]))
      ubs_qa_data.at[i-1,"question"]=question
      ubs_qa_data.at[i-1,"answer"]=answer
      ubs_qa_data.at[i-1, "analyst"]=analyst
      ubs_qa_data.at[i-1,"exective"]=exective
      question = str(ubs_qa_data["utterance"][i])
      answer = ""
      analyst = []
      exective = []
    else:
      continue


In [ ]:
#Check if the last of each segmented bank contains question and answer
#ubs_qa_data[10:60]
#print(ubs_qa_data)

               speaker job_title  \
0          Chis Hallam   Goldman   
1    Sergio P. Ermotti       UBS   
2         Chris Hallam   Goldman   
3     Kian Abouhossein  JPMorgan   
4    Sergio P. Ermotti       UBS   
..                 ...       ...   
360        Piers Brown      HSBC   
361       Todd Tuckner       UBS   
362  Sergio P. Ermotti       UBS   
363        Piers Brown      HSBC   
364  Sergio P. Ermotti       UBS   

                                             utterance        call_date  \
0    Chis Hallam, Goldman Sachs Yes. Good morning, ...    25 April 2023   
1    Okay. Thank you. On capital requirements, you ...    25 April 2023   
2                                  Very clear. Thanks.    25 April 2023   
3    Yeah. Thanks. Just two questions. The first on...    25 April 2023   
4    So, Sarah, take the first question. I'll take ...    25 April 2023   
..                                                 ...              ...   
360  21, okay. So in any case, to handle i

In [ ]:
"""
Adding multiple Analyst name and Exective name if more than 2 people are participating in the conversation
"""
for i in range(len(ubs_qa_data)):
  if isinstance(ubs_qa_data["exective"][i], list):
    ubs_qa_data["exective"][i]=list(set(ubs_qa_data["exective"][i]))
  else:
    continue


In [ ]:
"""
Dropping Nill in question columns to remove duplicated rows
"""
ubs_qa_data= ubs_qa_data.dropna(subset=["question"])
ubs_qa_data.reset_index(drop=True, inplace=True)

In [ ]:
#ubs_qa_data.head()

# **Cleaning Text and rearrangeing data frame:**
Cleaning each question and answer by removing stop words and symbols and lowering lettercase


In [ ]:
#preprocessing data
preprocessor(ubs_qa_data, "answer", "question_tokenised_data", "question_cleaned")
preprocessor(ubs_qa_data,"question","answer_tokenised_data","answer_cleaned")

<ipython-input-3-cd3cee0e7408>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col1]=data[col]
<ipython-input-3-cd3cee0e7408>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col2]=data[col]
<ipython-input-3-cd3cee0e7408>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [ ]:
#rename column
ubs_qa_data.rename(columns={"financial_quarter":"Quarter","question":"Question", "category":"Analyst_Bank", "answer": "Response", "exective":"Executive", "question_cleaned": "Question_cleaned", "answer_cleaned": "Response_cleaned", "source_file":"filename"},inplace=True)

In [ ]:
#reorganise column
ubs_qa_data=ubs_qa_data[["filename","Quarter","Question","Question_cleaned","Analyst_Bank","Response","Response_cleaned", "Executive"]]

In [ ]:
#Check the last dataframe
#ubs_qa_data.head()
print(ubs_qa_data)

               speaker job_title  \
0          Chis Hallam   Goldman   
1    Sergio P. Ermotti       UBS   
2         Chris Hallam   Goldman   
3     Kian Abouhossein  JPMorgan   
4    Sergio P. Ermotti       UBS   
..                 ...       ...   
360        Piers Brown      HSBC   
361       Todd Tuckner       UBS   
362  Sergio P. Ermotti       UBS   
363        Piers Brown      HSBC   
364  Sergio P. Ermotti       UBS   

                                             utterance        call_date  \
0    Chis Hallam, Goldman Sachs Yes. Good morning, ...    25 April 2023   
1    Okay. Thank you. On capital requirements, you ...    25 April 2023   
2                                  Very clear. Thanks.    25 April 2023   
3    Yeah. Thanks. Just two questions. The first on...    25 April 2023   
4    So, Sarah, take the first question. I'll take ...    25 April 2023   
..                                                 ...              ...   
360  21, okay. So in any case, to handle i

UBS Management Discussion

In [ ]:
ubs_manag_data = pd.read_csv("/content/drive/MyDrive/BOE/bank_of_england/data/cleansed/ubs_management_discussion.csv")
ubs_manag_data.head()

,speaker,utterance,call_date,financial_quarter,source_file
0,Unknown,"Youngwood, Group Chief Financial Officer Inclu...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
1,Sergio P. Ermotti,"Thank you, Sarah, good morning, everyone. I am...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
2,Latin America.,"In Asset Management, the combination will impr...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
3,Sarah Youngwood,"Thank you, Sergio. Good morning, everyone. rel...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
4,Group Chief Financial Officer,Including analyst Q&A session,7 May 2024,1Q24,1q24-earnings-call-remarks.pdf


In [ ]:
preprocessor( ubs_manag_data, "utterance", "question_tokenised_data", "Utterance_cleaned" )

In [ ]:
#Rename columns
ubs_manag_data.rename(columns={"financial_quarter":"Quarter","speaker":"Speaker","utterance":"Utterance"}, inplace=True)
#ubs_manag_data.head()

In [ ]:
#Reorder the columns
ubs_manag_data=ubs_manag_data[["Quarter","Speaker","Utterance","Utterance_cleaned"]]
ubs_manag_data.head()

,Quarter,Speaker,Utterance,Utterance_cleaned
0,1Q23,Unknown,"Youngwood, Group Chief Financial Officer Inclu...",[youngwood group chief financial officer inclu...
1,1Q23,Sergio P. Ermotti,"Thank you, Sarah, good morning, everyone. I am...",[thank you sarah good morning everyone happy b...
2,1Q23,Latin America.,"In Asset Management, the combination will impr...",[asset management combination improve position...
3,1Q23,Sarah Youngwood,"Thank you, Sergio. Good morning, everyone. rel...",[thank you sergio good morning everyone relate...
4,1Q24,Group Chief Financial Officer,Including analyst Q&A session,[including analyst q&a session]


# **Export the output as a csv file**

UBS QA section

In [ ]:
#export preprocessed data
#preprocessed_qa_csv_path3 = "/content/ubs_qa_df_preprocessed.csv"
#processed_df.to_csv(preprocessed_qa_csv_path3, index=False)

UBS management discussion

In [ ]:
#export preprocessed data
#preprocessed_qa_csv_path4 = "/content/sample_data/ubs_management_df_preprocessed.csv"
#manag_data.to_csv("ubs_management_discussion.csv", index=False)